In [17]:
local_model = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"
# Load the local LLM model
#llama_model = Llama(model_path=local_model, n_ctx=56000, verbose=False)
from llama_index.llms.llama_cpp import LlamaCPP
# Initialize LLM properly using LlamaCPP wrapper
llm = LlamaCPP(
    model_path=local_model,
    model_kwargs={
        "n_ctx": 2048,
        "n_threads": 4,
    }
)

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data with 26 key-value pairs and 579 tensors from /Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instru

In [18]:
# pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_path = "/Users/maxhuang/projects/llm/models/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbedding(model_name=embedding_model_path)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

In [2]:
 from SPARQLWrapper import SPARQLWrapper, JSON
 from rdflib import Graph
 import pandas as pd

In [3]:
def get_possible_entity_uris_from_wikidata(entity_name):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
    SELECT ?entity ?entityLabel WHERE {
        ?entity rdfs:label "%s"@en .
    } limit 5
    """ % entity_name)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    results = pd.json_normalize(results['results']['bindings']).values.tolist()
    results = ["<" + x[1] + ">" for x in results]
    return [*set(results)] # remove duplicates

In [4]:
def wikidata_query_to_df(entity_uri):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
       SELECT ?description ?is_a_type_of WHERE {
         %s schema:description ?description FILTER (lang(?description) = 'en') .
         %s wdt:P31 ?instanceOf .  
         ?instanceOf rdfs:label ?is_a_type_of FILTER (lang(?is_a_type_of) = 'en') .
       } limit 10
    """ % (entity_uri, entity_uri))

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results2 = pd.json_normalize(results['results']['bindings'])
    prompt_text = ""
    for index, row in results2.iterrows():
         prompt_text += row['description.value'] + " is a type of " + row['is_a_type_of.value'] + "\n" 
    return prompt_text

In [5]:
def generate_prompt_text(entity_name):
    entity_uris = get_possible_entity_uris_from_wikidata(entity_name)
    prompt_text = ""
    for entity_uri in entity_uris:
        p = wikidata_query_to_df(entity_uri)
        if "disambiguation page" not in p:
            prompt_text += entity_name + " is " + wikidata_query_to_df(entity_uri)
    return prompt_text

In [9]:
# print("Sedona:", generate_prompt_text("Sedona"))
# print("Donald Trump:", generate_prompt_text("Donald Trump"))

In [ ]:
from llama_index.core import TreeIndex
from llama_index.core import Document

def wd_query(question, *entity_names):
    prompt_texts = []
    for entity_name in entity_names:
        prompt_texts += [generate_prompt_text(entity_name)]
    documents = [Document(text=text) for text in prompt_texts]
    index = TreeIndex(documents)
    index = index.as_query_engine(child_branching_factor=2)
    return index.query(question)

In [20]:
print("Sedona:", wd_query("What is Sedona?", "Sedona"))


llama_print_timings:        load time =    6958.93 ms
llama_print_timings:      sample time =       1.03 ms /    21 runs   (    0.05 ms per token, 20447.91 tokens per second)
llama_print_timings: prompt eval time =    6958.68 ms /   114 tokens (   61.04 ms per token,    16.38 tokens per second)
llama_print_timings:        eval time =    2671.40 ms /    20 runs   (  133.57 ms per token,     7.49 tokens per second)
llama_print_timings:       total time =    9641.79 ms /   134 tokens


Sedona: 2012 film, city, company, 2015 single by Houndmouth
